## OpenSearch 한글 형태소 분석기를 통한 키워드 검색

### 1. 환경 세팅

In [1]:
%load_ext autoreload
%autoreload 2

import os,sys
module_path=".."
sys.path.append(os.path.abspath(module_path))

#### Bedrock Client 생성

In [2]:
from utils import bedrock
from utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region='us-east-1'
)


Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### Embedding Model 선택

In [3]:
from langchain.embeddings.bedrock import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
  client=boto3_bedrock,
  model_id='amazon.titan-embed-text-v1'
)

## 2. 데이터 준비

In [4]:
import pandas as pd
pd.options.display.max_rows = 20
data_file_path = 'data/fsi_smart_faq_ko.csv'
df = pd.read_csv(data_file_path)
df.head(3)

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


In [5]:
pre_df = df.drop(columns=['no'])
pre_df.rename(columns={"Category": "ask"}, inplace=True)
pre_df.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)
pre_df.head(3)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader로 문서 로딩

In [6]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(
  file_path="data/fsi_smart_faq_ko_preprocess.csv",
  source_column="Source",
  encoding='utf-8'
)
documents_fsi = loader.load()

In [7]:
print(documents_fsi[0])
'''
metadata={'source': '아마존은행', 'row': 0}
'''

page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 아마존은행' metadata={'source': '아마존은행', 'row': 0}


"\nmetadata={'source': '아마존은행', 'row': 0}\n"

### 문서의 metadata에 항목 추가
- column의 type, source는 metadta로 생성하고, 내용에서는 삭제.
- timestamp, embeddng model의 endpoint name을 metadata로 추가

In [8]:
import time
def creat_metadata(docs):
  # add a custom metadta field
  for idx, doc in enumerate(docs):
    split_content = doc.page_content.split("type: ")
    content = split_content[0]
    metadata = split_content[1]
    doc.metadata['type'] = metadata.split("\n")[0]
    doc.page_content = content # metadata 제외하고 content만 저장
    doc.metadata['timestamp'] = time.time()

creat_metadata(documents_fsi)

In [9]:
print(len(documents_fsi))
print(documents_fsi[0])

92
page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n' metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1701407810.8438127}


### Text Spliter로 chunking

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(
  chunk_size=2048,
  chunk_overlap=50,
  separators=["\n\n", "\n", ".", " ", ""],
  length_function=len,
)
docs = text_spliter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


## 4. OpenSearch Client 생성

In [11]:
aws_region='ap-northeast-2'
http_auth = ('admin', "Admin123@")
opensearch_domain_endpoint = "https://search-bedrock-fkik524xhgkrn3t2omdtfspzmm.ap-northeast-2.es.amazonaws.com"

from utils.opensearch import opensearch_utils

os_client = opensearch_utils.create_aws_opensearch_client(
  aws_region,
  opensearch_domain_endpoint,
  http_auth
)

## 4. OpenSearch 벡터 Indexer 생성
OpenSearch에 해당 인덱스가 존재하면, 삭제

In [12]:
index_name = 'genai-demo-index-v1'
index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
  opensearch_utils.delete_index(os_client, index_name)
else:
  print("index does not exist")

index_name=genai-demo-index-v1, exists=True

Deleting index:
{'acknowledged': True}


### 인덱스 생성

In [13]:
from langchain.vectorstores import OpenSearchVectorSearch
# by default, langchain would create a K-NN index and the embeddings would be ingested as a K-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
  index_name=index_name,
  documents=docs,
  embedding=llm_emb,
  opensearch_url=opensearch_domain_endpoint,
  http_auth=http_auth,
  bulk_size=1000,
  timeout=60
)

### 인덱스 확인

In [14]:
index_info = os_client.indices.get(index=index_name)
print(index_info)

{'genai-demo-index-v1': {'aliases': {}, 'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'}, 'source': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'timestamp': {'type': 'float'}, 'type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'text': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'vector_field': {'type': 'knn_vector', 'dimension': 1536, 'method': {'engine': 'nmslib', 'space_type': 'l2', 'name': 'hnsw', 'parameters': {'ef_construction': 512, 'm': 16}}}}}, 'settings': {'index': {'replication': {'type': 'DOCUMENT'}, 'number_of_shards': '5', 'knn.algo_param': {'ef_search': '512'}, 'provided_name': 'genai-demo-index-v1', 'knn': 'true', 'creation_date': '1701407849943', 'number_of_replicas': '1', 'uuid': 'Tx4AJRfoTROBvKJPy9UVtQ', 'version': {'created': '136327827'}}}}}


### 인덱싱 수정하기 (형태소 분석기 사용 enablement)

In [15]:
new_index_name = f"{index_name}-with-tokenizer"
new_index_name

'genai-demo-index-v1-with-tokenizer'

In [16]:
tokenizer = "nori"
analyzer_config = {
  "tokenizer": tokenizer,
  "tokenizer_type": f'{tokenizer}_tokenizer',
  "char_filter": ["html_strip"],
  "filter": ["nori_number", "nori_readingform", "lowercase"],
  "decompound_mode": "mixed",
  "discard_punctuation": True
}

In [17]:
index_info[index_name]["settings"]["analysis"] = {
  "tokenizer": {
    analyzer_config["tokenizer"]: {
      "type": analyzer_config["tokenizer_type"],
      "decompound_mode": analyzer_config["decompound_mode"],
      "discard_punctuation": analyzer_config["discard_punctuation"]
    }
  },
  "analyzer": {
    "my_analyzer": {
      "type": "custom",
      "tokenizer": analyzer_config["tokenizer"],
      "char_filter": analyzer_config["char_filter"],
      "filter": analyzer_config["filter"]
    }
  }
}

# Setting for Columns to be adapted by Tokenizer (tokenizer가 적용될 컬럼에 맞춰서 수정)
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]["mappings"]["properties"]["text"]["search_analyzer"] = "my_analyzer"

# Setting for vector index column (변경 없음)
index_info[index_name]["settings"]["index"] = {
    "number_of_shards": "5",
    "knn.algo_param": {"ef_search": "512"},
    "knn": "true",
    "number_of_replicas": "2"
}
del index_info[index_name]["aliases"]
new_index_info = index_info[index_name]

In [18]:
print(new_index_info)

{'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'}, 'source': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'timestamp': {'type': 'float'}, 'type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'text': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}, 'analyzer': 'my_analyzer', 'search_analyzer': 'my_analyzer'}, 'vector_field': {'type': 'knn_vector', 'dimension': 1536, 'method': {'engine': 'nmslib', 'space_type': 'l2', 'name': 'hnsw', 'parameters': {'ef_construction': 512, 'm': 16}}}}}, 'settings': {'index': {'number_of_shards': '5', 'knn.algo_param': {'ef_search': '512'}, 'knn': 'true', 'number_of_replicas': '2'}, 'analysis': {'tokenizer': {'nori': {'type': 'nori_tokenizer', 'decompound_mode': 'mixed', 'discard_punctuation': True}}, 'analyzer': {'my_analyzer': {'type': 'custom', 'tokenizer': 'nori', 'char_filter': ['html_strip'], 'filter': ['nori_nu

### 형태소 분석기용 인덱서 생성

In [19]:
index_exists = opensearch_utils.check_if_index_exists(os_client, new_index_name)
if index_exists:
    opensearch_utils.delete_index(os_client, new_index_name)
else:
    print("Index does not exist")

index_name=genai-demo-index-v1-with-tokenizer, exists=False
Index does not exist


In [20]:
opensearch_utils.create_index(
  os_client,
  index_name=new_index_name,
  index_body=new_index_info
)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-demo-index-v1-with-tokenizer'}


### Re-indexing

In [21]:
_reindex = {
  "source": {"index": index_name},
  "dest": {"index": new_index_name}
}
print("_reindex: \n", _reindex)

_reindex: 
 {'source': {'index': 'genai-demo-index-v1'}, 'dest': {'index': 'genai-demo-index-v1-with-tokenizer'}}


In [22]:
os_client.reindex(_reindex)

{'took': 1698,
 'timed_out': False,
 'total': 92,
 'updated': 0,
 'created': 92,
 'deleted': 0,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

## 6. 키워드 검색
"Text"에 "약관", 뱅킹" 단어를 검색
- without tokeninzer (index_name)

In [23]:
query = "뱅킹"
query = opensearch_utils.get_query(
  query=query
)
print("query: ", query)
resp = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(resp, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '뱅킹', 'minimum_should_match': '0%', 'operator': 'or'}}}], 'filter': []}}}
There is no response


- with tokenizer (new_index_name)

In [24]:
query = "뱅킹"
query = opensearch_utils.get_query(
  query=query
)
print("query: ", query)
resp = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(resp, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '뱅킹', 'minimum_should_match': '0%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  10
# of display: 3
---------------------
_id in index:  2314182f-eb5c-4686-b50a-6065bffd233b
1.4926369
ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)
{'source': '신한은행', 'row': 39, 'type': '인터넷뱅킹', 'timestamp': 1701407810.8439403}
---------------------
_id in index:  769c514b-6103-43bc-b46c-1b47412b8281
1.4263135
ask: 12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?
Information: 최근 12개월동안 인터넷뱅킹/폰뱅킹/모바일뱅킹에서 이체서비스를 이용하지 않은 경우 고객님의 금융자산 보호를 위하여 이체서비스가 중단됩니다. 다시 이체를 원하시는 경우에는 인터넷뱅킹/폰뱅킹/신한쏠(SOL)에

### 형태소 분석 결과 확인
"약관" 또는 "뱅킹" 확인
[중요]: doc_id: 위의 문서 인덱스 정보 확인 후 수정

In [25]:
doc_id = "769c514b-6103-43bc-b46c-1b47412b8281"

In [26]:
os_client.termvectors(index=index_name, id=doc_id, fields='text')

{'_index': 'genai-demo-index-v1',
 '_id': '769c514b-6103-43bc-b46c-1b47412b8281',
 '_version': 1,
 'found': True,
 'took': 153,
 'term_vectors': {'text': {'field_statistics': {'sum_doc_freq': 1021,
    'doc_count': 22,
    'sum_ttf': 1225},
   'terms': {'12개월': {'term_freq': 1,
     'tokens': [{'position': 1, 'start_offset': 5, 'end_offset': 9}]},
    '12개월동안': {'term_freq': 1,
     'tokens': [{'position': 10, 'start_offset': 58, 'end_offset': 64}]},
    '14세이상': {'term_freq': 1,
     'tokens': [{'position': 94, 'start_offset': 475, 'end_offset': 480}]},
    '1599': {'term_freq': 1,
     'tokens': [{'position': 113, 'start_offset': 567, 'end_offset': 571}]},
    '8000로': {'term_freq': 1,
     'tokens': [{'position': 114, 'start_offset': 572, 'end_offset': 577}]},
    'ars': {'term_freq': 2,
     'tokens': [{'position': 49, 'start_offset': 246, 'end_offset': 249},
      {'position': 68, 'start_offset': 352, 'end_offset': 355}]},
    'ask': {'term_freq': 1,
     'tokens': [{'position': 0

In [27]:
os_client.termvectors(index=new_index_name, id=doc_id, fields='text')

{'_index': 'genai-demo-index-v1-with-tokenizer',
 '_id': '769c514b-6103-43bc-b46c-1b47412b8281',
 '_version': 1,
 'found': True,
 'took': 33,
 'term_vectors': {'text': {'field_statistics': {'sum_doc_freq': 1562,
    'doc_count': 22,
    'sum_ttf': 2898},
   'terms': {'10014': {'term_freq': 1,
     'tokens': [{'position': 194, 'start_offset': 473, 'end_offset': 477}]},
    '12': {'term_freq': 2,
     'tokens': [{'position': 1, 'start_offset': 5, 'end_offset': 7},
      {'position': 20, 'start_offset': 58, 'end_offset': 60}]},
    '15998000': {'term_freq': 1,
     'tokens': [{'position': 232, 'start_offset': 567, 'end_offset': 576}]},
    '2': {'term_freq': 1,
     'tokens': [{'position': 147, 'start_offset': 360, 'end_offset': 361}]},
    'ars': {'term_freq': 2,
     'tokens': [{'position': 108, 'start_offset': 246, 'end_offset': 249},
      {'position': 144, 'start_offset': 352, 'end_offset': 355}]},
    'ask': {'term_freq': 1,
     'tokens': [{'position': 0, 'start_offset': 0, 'end_of

### Minimum_should_match 활용
query에 있는 단어의 n%이상 존재하는 문서만 가져온다

In [31]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=75
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '인터넷 뱅킹으로 예적금 해약', 'minimum_should_match': '75%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  3
# of display: 3
---------------------
_id in index:  9c66aab9-397e-411a-84de-ed9bac54f725
12.845038
ask: 인터넷 예적금 해약하려면 어떻게 해야 하나요?
Information: 인터넷에서 신규하셨고, 이후 통장발급을 받지 않으셨다면 인터넷뱅킹(http://bank.shinhan.com)의 금융상품 예금/신탁 해지 메뉴를 통해 해지하실 수 있습니다.
{'source': '신한은행', 'row': 70, 'type': '', 'timestamp': 1701407810.8440356}
---------------------
_id in index:  4ee7326f-b55e-48ad-9627-979423ab05ce
8.189483
ask: 인터넷으로 신규 예/적금 신청하는 방법을 알려주세요
Information: 인터넷상으로 예금/신탁을 신규가입하시려면 우선 고객님께서는인터넷뱅킹에 가입하셔야 하며 신규방법은 두 가지가 있습니다.1. 인터넷뱅킹에서 가입인터넷뱅킹 로그인을 하신 후 예금/신탁 > 신규 메뉴에서 예금 및 신탁 상품을 신규하실 수 있습니다.2. 신한S뱅크에서 가입신한S뱅크 상품센터 > 예금센터 메뉴에서 예금상품을 신규하실 수 있습니다.
{'source': '신한은행', 'row': 85, 'type': '인터넷뱅킹', 'timestamp': 1701407810.8440835}
---------------------
_id in index:  f742a43f-e704-4f77-af90-f70ddc3e83b8
6.601389
ask: 해외에서 예적금해지중 ARS 추가인증

minimum_should_match=100

In [32]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=100
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '인터넷 뱅킹으로 예적금 해약', 'minimum_should_match': '100%', 'operator': 'or'}}}], 'filter': []}}}
There is no response


### Filter 활용
- document내 metadata를 사용하여 search space를 줄일 수 있다.
- 특히 filter의 경우 search전에 수행되게 때문에, 검색 속도 향상을 기대할 수 있다.
- syntax
  - filter=[{"term"[고정]: {"metadata.source"[메타데이터 이름, 혹은 메타데이터 아니여도 상관없음]: "신한은행"[조건명]}},]
  - list 형식으로 복수개 filter 설정 가능

In [ ]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
  query=query,
  minimum_should_match=75,
  filter=[
    {"term": {"metadata.source": "신한은행"}},
    {"term": {"metadata.type": "인터넷뱅킹"}},
  ]
)
print("query: ", query)